In [0]:
%run /Workspaces/dars_nic_391419_j3w9t_collab/CCU013_02/atlas/00_Load_helpers

# Pivot HES APC

In [0]:
# Create a pivot version of HES APC 

import pyspark.sql.functions as f

archive = spark.table("dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive")

diag_fields = [c for c in archive.columns if c.startswith("DIAG_4") and "CONCAT" not in c]
diag_and_n = [(c, int(c.lstrip("DIAG_4"))) for c in diag_fields]

diag_columns = f.array(*(f.struct(f.col(c).alias("value"), f.lit(n).alias("n")) for c, n in diag_and_n))

apc_pivot = (
  archive
  .where("ProductionDate == '2022-01-20 14:58:52.353312'")
  .select("person_id_deid",
          f.col("epistart").alias("date"),
          f.explode(diag_columns).alias("diag_4"))
  .selectExpr("person_id_deid", "date", "diag_4.value as diag_4", "diag_4.n as n")
)

apc_pivot.createOrReplaceGlobalTempView('ccu013_apc_pivot')

In [0]:
apc_pivot.count()

In [0]:
drop_table("ccu013_apc_pivot")

In [0]:
create_table("ccu013_apc_pivot")

In [0]:
%sql
select * from dars_nic_391419_j3w9t_collab.ccu013_apc_pivot limit 5;